### 3.구글 플레이 스토어에서 앱 검색 후 댓글을 수집하는 코드를 작성하시오.

- 조건1: 내가 입력한 검색어를 기반으로 첫 번째 앱을 선택합니다.
- 조건2: 댓글 수집이 100개 이상 수집되어야 합니다. (댓글 갯수가 100미만인 앱은 예외)
- 조건3: 댓글에서 아래의 정보를 가져오세요.
    - 작성자명, 별표 갯수, 작성일, 작성 내용
- 조건4: 크롤링한 데이터를 오늘 날짜를 기준으로 엑셀에 저장하시오. (파일명: 2023-04-18)

In [1]:
import time
import pandas as pd

from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
url = 'https://play.google.com/store/apps'
browser = webdriver.Chrome()
browser.get(url)
browser.implicitly_wait(10)

## 앱 검색.
browser.find_element(By.XPATH, '//*[@id="kO001e"]/header/nav/div/div[1]/button/i').click()
browser.find_element(By.XPATH, '//*[@id="kO001e"]/header/nav/c-wiz/div/div/label/input').send_keys('speak' + Keys.ENTER)

## 앱 페이지 이동.
browser.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz[3]/div/div/c-wiz/c-wiz[1]/c-wiz/section/div/div/a').click()

In [3]:
## 리뷰 페이지.
## 스크롤을 밑으로 내리고 버튼을 클릭해야함...(심지어 클릭도 아님)

browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [4]:
## 리뷰 전체 보기 클릭
spread_review = browser.find_element(by=By.XPATH, value = '//*[@id="yDmH0d"]/c-wiz[4]/div/div/div[1]/div/div[2]/div/div[1]/div[1]/c-wiz[5]/section/div/div[2]/div[5]/div/div/button')
isTrue = spread_review.is_displayed()
if isTrue :
    browser.execute_script("arguments[0].click();", spread_review)
    time.sleep(1.5)

In [5]:
# 스크롤할 요소 찾기
all_reviews = browser.find_element(by=By.XPATH, value='//*[@id="yDmH0d"]/div[5]/div[2]/div/div/div/div/div[2]') ## 리뷰 컨테이너

# 이전 스크롤 높이
previous_height = browser.execute_script("return arguments[0].scrollHeight", all_reviews)

while True:
    # 요소 내부를 최하단으로 스크롤
    browser.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', all_reviews)
    
    # 페이지 로드를 기다림
    time.sleep(3)
    
    # 새로운 스크롤 높이
    new_height = browser.execute_script("return arguments[0].scrollHeight", all_reviews)
    
    # 스크롤 높이가 변경되지 않았다면, 더 이상 로드할 항목이 없는 것으로 판단하고 반복을 종료
    if new_height == previous_height:
        break
    
    # 이전 스크롤 높이 업데이트
    previous_height = new_height

In [6]:
total_reviews = browser.find_elements(By.CLASS_NAME, 'RHo1pe')
print(len(total_reviews))

2840


In [7]:
## 작성자명, 별표 갯수, 작성일, 작성 내용
total_data = []
for review in total_reviews:
    name = review.find_element(By.CLASS_NAME, 'X5PpBb').text
    stars = review.find_element(By.CLASS_NAME, 'iXRFPc').get_attribute('aria-label')
    date = review.find_element(By.CLASS_NAME, 'bp9Aid').text
    content = review.find_element(By.CLASS_NAME, 'h3YV2d').text

    star_str = stars.split(' ')
    max_star = star_str[1][0]
    min_star = star_str[3][0]
    # print(f"{name} {min_star}/{max_star} {date} {content}")
    total_data.append({'이름' : name, '평점' : f'{min_star}/{max_star}', '작성일' : date, '내용' : content})

In [8]:
current_time = datetime.now()
date_str = current_time.strftime('%Y-%m-%d')
filename = f'{date_str}.csv'


df = pd.DataFrame(total_data)
df.to_csv(filename)